In [1]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import pandas as pd
from openpyxl import load_workbook
import re
import io
import os

import sys
sys.path.append(os.path.abspath('../kev/'))

import eq_data
import eq_preproc
import eq_evaluator
import eq_postproc
import eq_writer


# basic input ------------------------------------------------
 
def cl_scripts_load(_sep, _subdir, _file):
    
    # if specific file selected it should be XLSX one
    if _file != "":
        
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        _file = _subdir + _file
        
        # open excel file
        with open(_file, "rb") as f:
            inmemory_file = io.BytesIO(f.read())
        wb = load_workbook(inmemory_file, read_only = True)
                
        r = re.compile(r'heats')
        heats_data = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], index_col = 0)
        
        r = re.compile(r'targets')
        targets = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0], header = None)
        
        r = re.compile(r'enthalpies')
        delta_H = pd.read_excel(_file, sheet_name = list(filter(r.search, wb.sheetnames))[0])
        
    # use a bunch of plain text files instead
    else:
          
        if _subdir != '':
            _subdir = '/' + _subdir
        _subdir = '../../input' + _subdir + '/'

        file_names = list(os.listdir(path = _subdir))

        r = re.compile(r'heats')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        heats_data = pd.read_csv(file, sep = _sep, decimal = ",", index_col = 0)
        
        r = re.compile(r'targets')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        targets = pd.read_csv(file, sep = _sep, header = None)
        
        r = re.compile(r'enthalpies')
        file = list(filter(r.search, file_names))[0]
        file = _subdir + str(file)
        delta_H = pd.read_csv(file, sep = _sep)
        
    return heats_data, targets, delta_H

In [2]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np
import re

# basic preprocessing ----------------------------------------
    
def cl_preproc(con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type):
    
    # checking if there are several series
    
    if 'series' not in con_data.columns:        
        con_data['series'] = ''
    
    if 'series' not in heats_data.index:
        heats_data.loc['series'] = ''
    
    if 'volumes' not in heats_data.index:
        heats_data.loc['volumes'] = 1        

    # series variables
    
    ser_info = con_data['series'].to_numpy()
    ser_unique = np.unique(ser_info)
    ser_num = np.shape(np.unique(ser_info))[0]
    
    ser_counts = con_data.groupby(['series']).size().tolist();
    
    # reading volumes from experimental data
    volumes = heats_data.loc['volumes'].to_numpy()
    
    # reading exp heats from experimental data
    heats = heats_data.loc['observation'].to_numpy() - heats_data.loc['dilution'].to_numpy()
        
    # transforming deviation to the matrix form
    devs = heats_data.loc['deviation'].to_numpy()
    devs = np.diag((1 / (np.array(devs) * np.array(devs))) * (np.sum(np.array(devs) * np.array(devs)) / len(devs)))
    
    # creating vector of known enthalpies
    dH_par = np.hstack((np.zeros(len(prod_names_con)), np.transpose(delta_H.drop('Reaction', axis = 1).to_numpy())[0]))
    
    # number of constant to find
    tar_names = set(targets.to_numpy()[0][1:])
    tar_num = [index for index, item in enumerate(prod_names) if item in tar_names]
    
    # number of enthalpy to find
    dH_names = np.hstack((prod_names_con, np.transpose(delta_H.drop('Value', axis = 1).to_numpy())[0]))
    un_el = set(prod_names) - set(dH_names)
    dH_ind_wtf = list([index for index, item in enumerate(prod_names) if item in un_el])
    un_el = set(prod_names) - (set(prod_names) - set(dH_names))
    dH_ind = list([index for index, item in enumerate(prod_names) if item in un_el])
    
    # creating array of diff total concentrations
    con_matrix_red = []
    for s in range(ser_num):
        for k in range(ser_counts[s] - 1):
            con_matrix_red.append(con_matrix[int(np.sum(ser_counts[:s])) + k + 1])
            
    if _calor_setup_type == 'DSC':
        _calor_setup_type_coeff = [[1] for i in range(len(con_matrix_red))]
    
    else:
        _calor_setup_type_coeff = 1 / np.array(con_matrix_red)[:, idx] 
    
    return (ser_num, ser_counts, ser_info, volumes, heats, devs, tar_num, dH_ind, dH_ind_wtf, dH_par, 
            _calor_setup_type_coeff)

In [3]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

import numpy as np

def cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff):

    # finding the increment in equilibrium concentrations of products - needed for dH calculation
    
    dif_conc = [] 
    for s in range(ser_num):    
        for k in range(ser_counts[s] - 1):
            dif_conc.append(c_res_out[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out[int(np.sum(ser_counts[:s])) + k])
               
    
    dif_conc, devs, volumes = np.array(dif_conc).astype(np.float32), devs.astype(np.float32), volumes.astype(np.float32)

    # finding vector of experimental values

    q_corr = heats
    for i in range(len(heats)):
        for j in range(len(volumes[dH_ind])):
            q_corr[i] -= volumes[dH_ind[j]] * dif_conc[i, dH_ind[j]] * dH_par[j]

    A = (volumes[dH_ind_wtf] * dif_conc[:, dH_ind_wtf]) * np.array(_calor_setup_type_coeff) 
    AT = np.transpose(A)
    LTP = np.linalg.inv(np.dot(np.dot(AT, devs), A))
    RTP = np.dot(np.dot(AT, devs), q_corr)
    dH_res = np.dot(LTP, RTP)
    dH_res_tmp = dH_par

    for i in range(len(dH_ind_wtf)):
        dH_res_tmp = np.insert(dH_res_tmp, dH_ind_wtf[i], dH_res[i])

    dH_res_out = dH_res_tmp

    F_min = 0
    for i in range(len(heats)):
        F_min += (np.sum(dH_res_out * dif_conc[i] * volumes[i] * np.array(_calor_setup_type_coeff[i])) - heats[i]) ** 2
    #print(sum(heats))
    #print(dH_res_out)
    #print(np.sum(dif_conc[i]))
    #print(np.sum(volumes[i]))
    #print(np.sum(np.array(_calor_setup_type_coeff[i])))
    
    # calculating st. dev. of dH_res / checking the quality of linear regression
    # consider making it in separate function
    
    disp = (F_min / (len(heats) - len(dH_ind_wtf)) * (np.diag(LTP))) ** 0.5  
    
    return F_min, dH_res_out, disp

In [4]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------

from copy import deepcopy
import eq_evaluator
# import also script cl_enthalpies when needed

# main Hooke-Jeeves procedure --------------------------------
def cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff):

    basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff)
    print(new_min)

    for i in range(max_iter):
        print('\niteration', i, lg_k[2])
        if F_min <= new_min:
        
            if step_init < HJ_eps:
                print("p1 " + str(step_init) + " " + str(HJ_eps))
                break
            
            else: 
                 
                step_init /= 2
            
                basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
                print('\nF min more ', new_min, F_min, lg_k[2])
                print(str(step_init))
        else:
            F_min = deepcopy(new_min)
            print('\nF min less', new_min, F_min, lg_k[2])
            print(str(step_init))
            basis_point_n = deepcopy(lg_k)
            lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])

            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                    con_matrix, ign_indices)

            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
            print('\nF min less after', new_min, F_min, lg_k[2])
        
            if F_min <= new_min:
                lg_k = deepcopy(basis_point_n)
                print('\nconstant less after', basis_point[tar_num], lg_k[2]) 
            else:
                F_min = new_min

            basis_point, lg_k, new_min, c_res_out, disp = cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num,
                                  st_coeff_matrix, type_con, con_matrix, ign_indices, ser_counts, ser_info, devs,
                                  volumes, heats, dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
        
            if F_min <= new_min:
            
                print('\nF min p2', new_min, F_min, lg_k[2])
                if step_init < HJ_eps:
                    print("p2")
                    break
            
                else: 
                
                    step_init /= 2
        
            else:

                F_min = deepcopy(new_min)
            
                print('\nF min p3.1', new_min, F_min, lg_k[2])
            
                basis_point_n = deepcopy(lg_k)
                lg_k[tar_num] = basis_point[tar_num] + 2 * (lg_k[tar_num] - basis_point[tar_num])
            
                c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
                                                        con_matrix, ign_indices)
    
                new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                              , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
        
                if F_min <= new_min:
                    lg_k = deepcopy(basis_point_n)
            
                print('\nF min p3.2', new_min, F_min, lg_k[2])
    
    return F_min, lg_k, dH_res_out, c_res_out, disp

# exploratory move function used in main function ------------------------
def cl_exploratory(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff):

    basis_point = deepcopy(lg_k)
    
    c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                con_matrix, ign_indices)

    new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                          , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)

    #if F_min > new_min:
    F_min = new_min
    
    for j in range(len(tar_num)):  
        print('input', lg_k[2], F_min)
        
        lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
        
        c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)
        #print('\na1 ', new_min, F_min)
        new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                   _calor_setup_type_coeff)
        print('right', lg_k[2], new_min, F_min)
        #print('\na2 ', new_min, F_min)
        if F_min <= new_min:
            
            lg_k[tar_num[j]] = lg_k[tar_num[j]] - 2 * step_init
            
            c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k, 
                                                        con_matrix, ign_indices)
        
            new_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats
                                                  , dH_ind, dH_par, dH_ind_wtf, _calor_setup_type_coeff)
            print('left', lg_k[2], new_min, F_min)
            #print('\na3 ', new_min, F_min)
            if F_min <= new_min:
                
                lg_k[tar_num[j]] = lg_k[tar_num[j]] + step_init
                new_min = deepcopy(F_min)
                #print('\na4 ', new_min, F_min)
            #else:
                
                #new_min = deepcopy(F_min)
                
        #else:
            
            #new_min = deepcopy(F_min)
    print('return', basis_point[2], lg_k[2])
    #print('\na5 ', new_min, F_min)
    return basis_point, lg_k, new_min, c_res_out, disp

In [5]:
# ########################################################## #
#                                                            #
# Name: KEV:Constant Evaluator                               #
# Author: GGamov                                             #
# Date: 2019                                                 #
#                                                            #
# ########################################################## #

# import libraries -------------------------------------------
import numpy as np
import eq_evaluator
# import also script cl_enthalpies when needed

def cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                      ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                      _calor_setup_type_coeff):
    
    # creating constants vectors ---------------------------------
    lg_k_1, lg_k_2 = np.array(lg_k), np.array(lg_k)
    heats_one, heats_two = [0] * len(heats), [0] * len(heats)
    jc_mtrx = [[0] * len(heats) for i in range(len(tar_num))]

    for i in range(len(tar_num)):
        # calculating the heats for Jacobi matrix computing
        lg_k_1[tar_num[i]] += HJ_eps
        lg_k_2[tar_num[i]] -= HJ_eps
    
        # if lg_k is overestimated
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_1,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])
    
        for o in range(len(heats)):
            heats_one[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
        # if lg_k is underestimated    
        c_res_out_dist, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k_2,
                                                        con_matrix, ign_indices)
    
        new_min, dH_res_out_dist, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                                                       _calor_setup_type_coeff)
    
        dif_conc = [] 
        for s in range(ser_num):    
            for k in range(ser_counts[s] - 1):
                dif_conc.append(c_res_out_dist[int(np.sum(ser_counts[:s])) + k + 1] - c_res_out_dist[int(np.sum(ser_counts[:s])) + k])    
   
        for o in range(len(heats)):
            heats_two[o] = np.sum(dif_conc[o] * dH_res_out_dist) * volumes[o]
    
    # calculating the Jacobi matrix    
    jc_mtrx[i] = (np.array(heats_one) - np.array(heats_two)) / (2 * HJ_eps)

    jc_mtrx, devs = np.array(jc_mtrx).astype(np.float32), devs.astype(np.float32)

    cov_matrix = (F_min / (len(heats) - len(tar_num))) * np.linalg.inv(np.dot(np.dot(jc_mtrx, devs), np.transpose(jc_mtrx)))

    errs_values = np.diag(cov_matrix) ** 0.5
    return errs_values

In [6]:
# input for xlsx file
_subdir = "calorimetry"
_sep = ";"
_file = "test_2.xlsx"
_calor_setup_type = 'DSC' # use 'DSC' for test_1 and test_2; use 'ampoule' for test_3

max_iter, eps, HJ_eps, step_init = 100, 0.0000001, 0.0001, 0.5

# load data

st_coeff_data, lg_k_data, con_data, type_con, component_name_for_yields = eq_data.eq_scripts_load(_sep, _subdir, _file)
heats_data, targets, delta_H = cl_scripts_load(_sep, _subdir, _file)

# preproc

st_coeff_matrix, prod_names, lg_k, prod_names_con, con_matrix, ign_indices, idx, type_con = eq_preproc.eq_preproc(
    st_coeff_data, con_data, type_con, lg_k_data, component_name_for_yields)

(ser_num, ser_counts, ser_info, volumes, heats, devs, 
 tar_num, dH_ind, dH_ind_wtf, dH_par, _calor_setup_type_coeff) = cl_preproc(
    con_data, prod_names, idx, type_con, heats_data, targets, delta_H, _calor_setup_type)

#----
lg_k[tar_num] = 10 # it is just for quick variation of initial approximation of the constant to be optimized. Delete in the end
#----

# calculation of eq concentration (starting point)
c_res_out, g_res_out = eq_evaluator.eq_calc(max_iter, eps, st_coeff_matrix, type_con, lg_k,
            con_matrix, ign_indices)

# calculation of F_min and unknown enthalpies (starting point)
F_min, dH_res_out, disp = cl_enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               _calor_setup_type_coeff)

# F_min assignment for further calculations
new_min = F_min

# constant refinment
if len(tar_num) > 0:
    F_min, lg_k, dH_res_out, c_res_out, disp = cl_hooke_jeeves(F_min, tar_num, lg_k, step_init, max_iter, eps, ser_num, st_coeff_matrix, type_con,
        con_matrix, ign_indices, ser_counts, ser_info, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                _calor_setup_type_coeff)

# error estimation
if len(tar_num) > 0:
    errs_values = cl_constants_errs(lg_k, heats, tar_num, HJ_eps, max_iter, eps, st_coeff_matrix, type_con, con_matrix,
                                    ign_indices, ser_num, ser_counts, devs, volumes, dH_ind, dH_par, dH_ind_wtf, 
                                    _calor_setup_type_coeff)

Check the consistency of reagent names!
[[7.29400000e-04 1.00000000e-18]
 [7.28186501e-04 7.28015934e-05]
 [7.26977034e-04 1.45361350e-04]
 [7.25771578e-04 2.17680473e-04]
 [7.24570112e-04 2.89760157e-04]
 [7.23372618e-04 3.61601589e-04]
 [7.22179076e-04 4.33205950e-04]
 [7.20989466e-04 5.04574409e-04]
 [7.19803768e-04 5.75708132e-04]
 [7.18621964e-04 6.46608274e-04]
 [7.17444034e-04 7.17275983e-04]
 [7.16269960e-04 7.87712402e-04]
 [7.15099722e-04 8.57918664e-04]
 [7.13933301e-04 9.27895895e-04]
 [7.12770680e-04 9.97645213e-04]
 [7.11611839e-04 1.06716773e-03]
 [7.10456760e-04 1.13646455e-03]
 [7.09305425e-04 1.20553678e-03]
 [7.08157815e-04 1.27438549e-03]
 [7.07013913e-04 1.34301178e-03]]
The component name for partition should be among those of basis components
input [10.] 0.001495102021050471
right [10.5] 0.0015008373792504504 0.001495102021050471
left [9.5] 0.0014844301975992435 0.001495102021050471
return [10.] [9.5]
0.0014844301975992435

iteration 0 [9.5]

F min less 0.0014844

In [7]:
print(np.array(c_res_out))
#print(c_res_out[int(np.sum(ser_counts[:0])) + 1])

[[7.29400000e-04 1.33604678e-20 9.86639543e-19]
 [6.56464037e-04 1.07912854e-06 7.17224657e-05]
 [5.84033131e-04 2.41744755e-06 1.42943904e-04]
 [5.12209291e-04 4.11818659e-06 2.13562289e-04]
 [4.41155367e-04 6.34541142e-06 2.83414748e-04]
 [3.71144694e-04 9.37366521e-06 3.52227928e-04]
 [3.02663548e-04 1.36904219e-05 4.19515532e-04]
 [2.36632245e-04 2.02171887e-05 4.84357225e-04]
 [1.74873924e-04 3.07782874e-05 5.44929850e-04]
 [1.20863243e-04 4.88495526e-05 5.97758728e-04]
 [7.94652541e-05 7.92972033e-05 6.37978791e-04]
 [5.27633475e-05 1.24205789e-04 6.63506638e-04]
 [3.71954690e-05 1.80014408e-04 6.77904318e-04]
 [2.80002527e-05 2.41962846e-04 6.85933049e-04]
 [2.22111229e-05 3.07085656e-04 6.90559557e-04]
 [1.83157666e-05 3.73871659e-04 6.93296072e-04]
 [1.55444993e-05 4.41552293e-04 6.94912261e-04]
 [1.34834135e-05 5.09714765e-04 6.95822011e-04]
 [1.18954661e-05 5.78123142e-04 6.96262349e-04]
 [1.06368811e-05 6.46634748e-04 6.96377032e-04]]


In [8]:
print('\nStoich coeff data')
print(st_coeff_data)

print('\nlg K')
print(lg_k_data)

print('\nconcentrations')
print(con_data)

print('\ntype con')
print(type_con)

print('\nexperimental data')
print(heats_data)

print('\ntargets')
print(targets)

print('\ndelta_H')
print(delta_H)


Stoich coeff data
   PLP  MF3H  name
0    1     1  Comp

lg K
   lg_k
0  5.15

concentrations
         PLP           T3H series
0   0.000729  1.000000e-18       
1   0.000728  7.280159e-05       
2   0.000727  1.453614e-04       
3   0.000726  2.176805e-04       
4   0.000725  2.897602e-04       
5   0.000723  3.616016e-04       
6   0.000722  4.332059e-04       
7   0.000721  5.045744e-04       
8   0.000720  5.757081e-04       
9   0.000719  6.466083e-04       
10  0.000717  7.172760e-04       
11  0.000716  7.877124e-04       
12  0.000715  8.579187e-04       
13  0.000714  9.278959e-04       
14  0.000713  9.976452e-04       
15  0.000712  1.067168e-03       
16  0.000710  1.136465e-03       
17  0.000709  1.205537e-03       
18  0.000708  1.274385e-03       
19  0.000707  1.343012e-03       

type con
0    tot
1    tot
Name: 0, dtype: object

experimental data
                   1         2         3         4        5         6   \
data                                           

In [9]:
print('\nSeries')
print(ser_num)
print(ser_counts)
print(ser_info)

print('\nSt coeff matr')
print(st_coeff_matrix)

print('\nprod names')
print(prod_names)
print(prod_names_con)

print('\nconst')
print(lg_k)

print('\nconcentrations')
print(con_matrix)

print(ign_indices)

print('\ntype con')
print(type_con)

print('\nvolumes')
print(volumes)

print('\nheats')
print(heats)

print('\ndeviations')
print(devs)

print('\nwhat to find and what is known')
print(tar_num)
print(dH_ind)
print(dH_ind_wtf)

print('\nenthalpies')
print(dH_par)


Series
1
[20]
['' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '' '']

St coeff matr
[[1 0]
 [0 1]
 [1 1]]

prod names
['PLP', 'T3H', 'Comp']
['PLP', 'T3H']

const
[[0.        ]
 [0.        ]
 [5.00543213]]

concentrations
[[7.29400000e-04 1.00000000e-18]
 [7.28186501e-04 7.28015934e-05]
 [7.26977034e-04 1.45361350e-04]
 [7.25771578e-04 2.17680473e-04]
 [7.24570112e-04 2.89760157e-04]
 [7.23372618e-04 3.61601589e-04]
 [7.22179076e-04 4.33205950e-04]
 [7.20989466e-04 5.04574409e-04]
 [7.19803768e-04 5.75708132e-04]
 [7.18621964e-04 6.46608274e-04]
 [7.17444034e-04 7.17275983e-04]
 [7.16269960e-04 7.87712402e-04]
 [7.15099722e-04 8.57918664e-04]
 [7.13933301e-04 9.27895895e-04]
 [7.12770680e-04 9.97645213e-04]
 [7.11611839e-04 1.06716773e-03]
 [7.10456760e-04 1.13646455e-03]
 [7.09305425e-04 1.20553678e-03]
 [7.08157815e-04 1.27438549e-03]
 [7.07013913e-04 1.34301178e-03]]
[]

type con
0    tot
1    tot
Name: 0, dtype: object

volumes
[15.024997 15.049994 15.074991 15.099988 15.12

In [10]:
print('\nEq conc')
print(np.array(c_res_out))
print(np.shape(c_res_out))


Eq conc
[[7.29400000e-04 1.33604678e-20 9.86639543e-19]
 [6.56464037e-04 1.07912854e-06 7.17224657e-05]
 [5.84033131e-04 2.41744755e-06 1.42943904e-04]
 [5.12209291e-04 4.11818659e-06 2.13562289e-04]
 [4.41155367e-04 6.34541142e-06 2.83414748e-04]
 [3.71144694e-04 9.37366521e-06 3.52227928e-04]
 [3.02663548e-04 1.36904219e-05 4.19515532e-04]
 [2.36632245e-04 2.02171887e-05 4.84357225e-04]
 [1.74873924e-04 3.07782874e-05 5.44929850e-04]
 [1.20863243e-04 4.88495526e-05 5.97758728e-04]
 [7.94652541e-05 7.92972033e-05 6.37978791e-04]
 [5.27633475e-05 1.24205789e-04 6.63506638e-04]
 [3.71954690e-05 1.80014408e-04 6.77904318e-04]
 [2.80002527e-05 2.41962846e-04 6.85933049e-04]
 [2.22111229e-05 3.07085656e-04 6.90559557e-04]
 [1.83157666e-05 3.73871659e-04 6.93296072e-04]
 [1.55444993e-05 4.41552293e-04 6.94912261e-04]
 [1.34834135e-05 5.09714765e-04 6.95822011e-04]
 [1.18954661e-05 5.78123142e-04 6.96262349e-04]
 [1.06368811e-05 6.46634748e-04 6.96377032e-04]]
(20, 3)


In [11]:
print('\nlg K')
print(lg_k)

print('\nF_min')
print(F_min)

print('\nFinal enthalpies')
print(dH_res_out)

print('\ncalculated error of enthalpy')
print(disp)

if len(tar_num) > 0:
    print('\ncalculated error of constant')
    print(errs_values)


lg K
[[0.        ]
 [0.        ]
 [5.00543213]]

F_min
0.00031194172710105714

Final enthalpies
[0.0 0.0 50.50237173832931]

calculated error of enthalpy
[1.33820058]

calculated error of constant
[0.13687211]


In [12]:
#import matplotlib.pyplot

#s = 4.5

#xxx = []
#FFF = []
#while s < 5.5:
    #lg_k[tar_num] = s
    
    #c_res_out = eq_calc(max_iter, eps, ser_num, st_coeff_matrix, type_con, lg_k,
        #con_matrix, ign_indices, ser_counts, ser_info)
        
    #F_min, dH_res_out = enthalpies(ser_num, ser_counts, c_res_out, devs, volumes, heats, dH_ind, dH_par, dH_ind_wtf, 
                               #_calor_setup_type_coeff)
    
    #xxx.append(s)
    #FFF.append(F_min)
    #s += 0.02


#matplotlib.pyplot.plot(xxx, FFF, 'ro-')

#import sys
#import os
##sys.path.append('../kev_calorim')
#print(os.path.abspath('../kev/'))
#print(os.getcwd())

#lg_k is array